# C9 Data Science Capstone Project
## Week 3 Graded Assignment
### 1.1 Import the necessary libraries

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

'''
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
'''

from bs4 import BeautifulSoup

### 1.2. Method for Downloading the City Information

In [2]:
def download_city_info(city_url):
    response = requests.get(city_url)
    assert response.status_code == 200  #HTTP 200 OK success status response code indicates that the request has succeeded. 
    return response.text

### 1.3. Method for Converting a City's Data from HTML into a DataFrame

In [3]:
def city2df(city_url):

    html_city = download_city_info(city_url)

    soup = BeautifulSoup(html_city, 'html.parser')

    city_table = []
    parser=soup.find('table')
             
    for td in parser.find_all('td'):
        cell = {}
        if td.span.text=='Not assigned': # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
            pass
        else:
            cell['PostalCode'] = td.select("p > b")[0].string
            cell['Borough'] = (td.span.text).split('(')[0] #td.select("p a:nth-of-type(1)")[0].get('title')
            cell['Neighborhood'] = (((((td.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            city_table.append(cell)
    
    df=pd.DataFrame(city_table)
        
    return df

### 1.4. Run the city2df method for acquiring the Toronto's data and placing it into a DataFrame

In [4]:
city_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #M for Toronto

df = city2df(city_url)

# Cleanse the data:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                 'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                 'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                 'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.shape

(103, 3)